## Imports

In [1]:
import math
from src.model_evaluation import load_model, evaluate_model, generate_text
from src.model_training import RNNLanguageModel, LSTMLanguageModel
import numpy as np
import os
import pickle
import random
import torch
import torch.nn as nn

## Setup

In [2]:
# Device agnostic code
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set seeds
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False
random.seed(seed)
np.random.seed(seed)

## Load Data

In [3]:
with open(os.path.normpath(os.path.join("data", "word_tokenisation_reuters_data.pkl")), "rb") as f:
    word_tokenised_numericalised_docs = pickle.load(f)

with open(os.path.normpath(os.path.join("data", "word_tokenisation_reuters_train_vocab.pkl")), "rb") as f:
    word_tokenisation_train_vocab = pickle.load(f)

with open(os.path.normpath(os.path.join("data", "subword_tokenisation_reuters_data.pkl")), "rb") as f:
    subword_tokenised_numericalised_docs = pickle.load(f)

with open(os.path.normpath(os.path.join("data", "subword_tokenisation_reuters_train_vocab.pkl")), "rb") as f:
    subword_tokenisation_train_vocab = pickle.load(f)

## Set Hyperparameters

In [4]:
EMBEDDING_SIZE = 128
HIDDEN_SIZE = 256
NUM_LAYERS = 2
SEQUENCE_LENGTH = 32
BATCH_SIZE = 32

## RNN

### Word Tokenisation, Dropout = 0.0

In [5]:
# Initialise model
rnn = RNNLanguageModel(
    vocab_size=len(word_tokenisation_train_vocab),
    embed_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    dropout=0.0,
    pad_idx=word_tokenisation_train_vocab["<pad>"]
).to(device)

rnn_word_tokenised_dropout_0_filename = "model_weights_rnn_word_tokens_True_dropout_0.0_20250905-091637"

rnn_word_tokenised_dropout_0 = load_model(model_initialised=rnn, filename=rnn_word_tokenised_dropout_0_filename, device=device)

In [6]:
# Cross Entropy Loss
rnn_word_tokenised_dropout_0_cross_entropy_loss = evaluate_model(
    model=rnn_word_tokenised_dropout_0,
    converted_tokenised_docs=word_tokenised_numericalised_docs,
    seq_len=SEQUENCE_LENGTH,
    batch_size=BATCH_SIZE,
    criterion=nn.CrossEntropyLoss(ignore_index=word_tokenisation_train_vocab["<pad>"]),
    device=device,
    vocab_size=len(word_tokenisation_train_vocab)
)
print(f"Cross Entropy Loss: {rnn_word_tokenised_dropout_0_cross_entropy_loss}")

# Perplexity
rnn_word_tokenised_dropout_0_perplexity = math.exp(rnn_word_tokenised_dropout_0_cross_entropy_loss)
print(f"Perplexity: {rnn_word_tokenised_dropout_0_perplexity}")

100%|██████████| 1073/1073 [01:22<00:00, 13.00it/s]


Cross Entropy Loss: 10.025398466093787
Perplexity: 22593.069209792488


In [7]:
# Generate text
starting_sequence = ["Today"]
temperatures = [0.7, 1.0, 1.3]

for temperature in temperatures:
    rnn_word_tokenised_dropout_0_generated_sequence = generate_text(
        model=rnn_word_tokenised_dropout_0,
        train_vocab=word_tokenisation_train_vocab,
        start_seq=starting_sequence,
        temperature=temperature
    )
    print(f"Generated Sequence from {starting_sequence} with temperature {temperature}:")
    print(rnn_word_tokenised_dropout_0_generated_sequence)
    print()

100%|██████████| 300/300 [00:02<00:00, 133.47it/s]


Generated Sequence from ['Today'] with temperature 0.7:
Today . clark said that if the slow fell for 97 . 6 . 3 billion vs 122 . 3 billion deposits 22 . 01 domes use 24 shr 33 cts vs 18 cts net 220 , 000 vs 1 , 250 , 000 assets 1 . 09 billion vs 861 . 2 mln deposits 817 . 7 mlnvs 705 . 7 mln loans 704 . 1 mln vs 553 . 4 mln nine mths shr 1 . 67 dlrs vs 2 . 30 dlrs net 34 . 0 mln vs 34 . 6 mln assets 2 . 7 billion vs 21 . 3 billion note 1987 qtr and nine mths includes gain 6 , 016 , 000 dlrs and 18 . 7 mln dlrs , or 56 cts a share vs 6 . 10 mln last month . e . c . 1986/87 coarse grain production is projected at 81 . 22 mln tonnes , vs 81 . 19 mln tonnes last month . the department said the provision for loan capital , unchanged from operating and alloy , in which societe generale de belgique &lt belb . br> holds an indirect 52 pct stake , said a sharp at 100 pct of the group of other milds coffee-growing nations will hold talks in guatemala next month to map their strategy for next sep

100%|██████████| 300/300 [00:01<00:00, 155.28it/s]


Generated Sequence from ['Today'] with temperature 1.0:
Today rejected its offer to only a minimum record date on a positive brokerage firm ships had been the spread . these ships were made if dome ends worries into the two monetary course , vice-president helmut schlesinger told reuters in the june m-1 the measures will affect us to prevent them from technological force to be a u . s . agriculture secretary richard lyng , who spoke the weekly national production in june , it said . there is no in paris recently , is conditioned on receipt of at least exports above the five billion dlrs of 182-day bills were priced at an average 96 . 488 dlrs against 96 , 000 tonnes of output ( fabk ) , ) from edmonton/swann hills 1 , 111 , 000 ) , analysts said . in state minerals , who get a net 1 . 2 mln units ( 50 mln barrels on march 16 central note port yesterday ' s issued of the company . according to its u . s . video vending shares . pursuant to the transaction , harvey seslowsky and william 

100%|██████████| 300/300 [00:02<00:00, 143.82it/s]

Generated Sequence from ['Today'] with temperature 1.3:
Today for protection in the legal and marketing economic growth would become large at another 50 , effective yesterday . the increase brings conoco rose a 0 . 5 pct stake in scandinavia fund first boston ltd ( 25 , 000 tonnes of the week involved from first tuskaloosa . the company said consumer prices full 12 mln from the 96 mln shares of &lt banque industrielle et mobiliere privee> ( bimp ) ' s remarks an average of holdings from 50 dlrs per share of southmark said it planned to sell its tire business to the amended offer market which would reduce competition , all ec officials have entered , several special zealand ltd , said monsanto in u . k . september m3 rises 0 . 8 nil nil o . o . p . -d 25 . 0 e . the housing group dropped 33 . 9 mln vs 78 . 9 mln nine mths shr 1 . 96 dlrs vs 1 . 23 dlrs net 6 , 240 , 939 dlrs vs 989 , 714 dlrs revs 32 . 9 mln vs 13 . 1 mln . after charge-offs of japan governor satoshi sumita told a press

### Word Tokenisation, Dropout = 0.2

In [8]:
# Initialise model
rnn = RNNLanguageModel(
    vocab_size=len(word_tokenisation_train_vocab),
    embed_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    dropout=0.2,
    pad_idx=word_tokenisation_train_vocab["<pad>"]
).to(device)

rnn_word_tokenised_dropout_02_filename = "model_weights_rnn_word_tokens_True_dropout_0.2_20250906-065216"

rnn_word_tokenised_dropout_02 = load_model(model_initialised=rnn, filename=rnn_word_tokenised_dropout_02_filename, device=device)

In [9]:
# Cross Entropy Loss
rnn_word_tokenised_dropout_02_cross_entropy_loss = evaluate_model(
    model=rnn_word_tokenised_dropout_02,
    converted_tokenised_docs=word_tokenised_numericalised_docs,
    seq_len=SEQUENCE_LENGTH,
    batch_size=BATCH_SIZE,
    criterion=nn.CrossEntropyLoss(ignore_index=word_tokenisation_train_vocab["<pad>"]),
    device=device,
    vocab_size=len(word_tokenisation_train_vocab)
)
print(f"Cross Entropy Loss: {rnn_word_tokenised_dropout_02_cross_entropy_loss}")

# Perplexity
rnn_word_tokenised_dropout_02_perplexity = math.exp(rnn_word_tokenised_dropout_02_cross_entropy_loss)
print(f"Perplexity: {rnn_word_tokenised_dropout_02_perplexity}")

100%|██████████| 1073/1073 [02:01<00:00,  8.83it/s]


Cross Entropy Loss: 8.077124800397653
Perplexity: 3219.9618814122496


In [10]:
# Generate text
starting_sequence = ["Today"]
temperatures = [0.7, 1.0, 1.3]

for temperature in temperatures:
    rnn_word_tokenised_dropout_02_generated_sequence = generate_text(
        model=rnn_word_tokenised_dropout_02,
        train_vocab=word_tokenisation_train_vocab,
        start_seq=starting_sequence,
        temperature=temperature
    )
    print(f"Generated Sequence from {starting_sequence} with temperature {temperature}:")
    print(rnn_word_tokenised_dropout_02_generated_sequence)
    print()

100%|██████████| 300/300 [00:01<00:00, 176.27it/s]


Generated Sequence from ['Today'] with temperature 0.7:
Today . it denied that the budget plan already successful and would like to buy burlington , that burlington has so far this year . but that experts ' prediction for economic growth this year , burnell roberts , chairman and chief executive officer , revamped the company ' s royalty plan , which includes wells fargo security guards , and chilton corp , a credit rating service . borg-warner has been reported to be acquired shares of common stock . the funds also said lynch said its u . k . video vending shares . pursuant to the transaction , harvey seslowsky and william hodes resigned from china fell 2 . 80 pct of the high crop improvement from profitable oil , traders said . in its semi-annual review of the world economy , the oecd said that apparent disagreements among major countries than at this time . if he was content of reserves before the imf and , the president of the meeting is at the new york law said . the agriculture d

100%|██████████| 300/300 [00:01<00:00, 180.87it/s]


Generated Sequence from ['Today'] with temperature 1.0:
Today repeated the offer this year as it is based on by reporters for the year end as above those reported a net loss of 976 . 8 mln dlrs for the year ending last december . traders said the story , which appeared by the danish foreign investment in u . k . manufacturing farm consumption will be the dealer-manager for the offer , which expires at midnight edt may 8 planned to extended when the 500 bought a tonne in the current fiscal year . it said the purchase price was 240 mln dlrs , as previously announced , the money stock is tender . to less than 20 pct . the unimedia , which has an asking general agreement to use 2 . 3 pct down , liquefied cost of may rise to 3 . 1 pct . at this week ' s imf owned by june four opec worldwide in the year ending december 1986 , with extended , the reserve said . the bank ' s bill rank so far was the most likely source of harcourt ' s weekly growth bank of nine major nations it are still worrie

100%|██████████| 300/300 [00:01<00:00, 207.17it/s]

Generated Sequence from ['Today'] with temperature 1.3:
Today says for each series of grains programs announced last year before investment operations totals around 500 mln dlrs . data reported compared with a 15 mln dlr but before a rise , compared on a new series of foreign-targeted shares , was set in august after june 26 and over 60 mln dlrs before taxes for high grade pfennig for five pct owned by exxon corp ' s firm j . 3 who rose to 96 . arrangements for a price and modification of 54 , 000 tonnes of gatt this year was not discussed in detail . japan sees the ec proposal required again to an 160 mln 5 . 0 in april last month , down two pct from the comparable 1985 quarter , up 35 pct from the wood . products crop seasonally adjusted 11 . 12 billion stg - both actual . and new oil production to around 16 dlrs a barrel . unocal said its market decision on here , they added . london metal exchange , a yeutter filed by merrill lynch capital partners , said it would continue its tend

### Subword Tokenisation, Dropout = 0.0

In [11]:
# Initialise model
rnn = RNNLanguageModel(
    vocab_size=len(subword_tokenisation_train_vocab),
    embed_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    dropout=0.0,
    pad_idx=subword_tokenisation_train_vocab["<pad>"]
).to(device)

rnn_subword_tokenised_dropout_0_filename = "model_weights_rnn_word_tokens_False_dropout_0.0_20250906-074545"

rnn_subword_tokenised_dropout_0 = load_model(model_initialised=rnn, filename=rnn_subword_tokenised_dropout_0_filename, device=device)

In [12]:
# Cross Entropy Loss
rnn_subword_tokenised_dropout_0_cross_entropy_loss = evaluate_model(
    model=rnn_subword_tokenised_dropout_0,
    converted_tokenised_docs=subword_tokenised_numericalised_docs,
    seq_len=SEQUENCE_LENGTH,
    batch_size=BATCH_SIZE,
    criterion=nn.CrossEntropyLoss(ignore_index=subword_tokenisation_train_vocab["<pad>"]),
    device=device,
    vocab_size=len(subword_tokenisation_train_vocab)
)
print(f"Cross Entropy Loss: {rnn_subword_tokenised_dropout_0_cross_entropy_loss}")

# Perplexity
rnn_subword_tokenised_dropout_0_perplexity = math.exp(rnn_subword_tokenised_dropout_0_cross_entropy_loss)
print(f"Perplexity: {rnn_subword_tokenised_dropout_0_perplexity}")

100%|██████████| 1442/1442 [02:35<00:00,  9.26it/s]


Cross Entropy Loss: 8.542993121851499
Perplexity: 5130.678147131693


In [13]:
# Generate text
starting_sequence = ["Today"]
temperatures = [0.7, 1.0, 1.3]

for temperature in temperatures:
    rnn_subword_tokenised_dropout_0_generated_sequence = generate_text(
        model=rnn_subword_tokenised_dropout_0,
        train_vocab=subword_tokenisation_train_vocab,
        start_seq=starting_sequence,
        temperature=temperature
    )
    print(f"Generated Sequence from {starting_sequence} with temperature {temperature}:")
    print(rnn_subword_tokenised_dropout_0_generated_sequence)
    print()

  0%|          | 0/300 [00:00<?, ?it/s]

100%|██████████| 300/300 [00:01<00:00, 194.20it/s]


Generated Sequence from ['Today'] with temperature 0.7:
Today ," ▁said ▁fil des . 
 ▁he ▁said ▁sq ui bb ▁was ▁the ▁good ▁partner ▁because ▁sq ui bb ▁has ▁a 
 ▁major ▁joint ▁venture ▁in ▁j apan ▁and ▁will ▁be 
 ▁complete ▁by ▁the ▁pipeline ▁break ▁and ▁the ▁company 's ▁new ▁preference ▁but ▁will ▁draw ▁all ▁the ▁amount 
 ▁rate ▁to ▁down ▁the ▁n ether lands ▁ant illes ▁and 
 ▁ar uba . ▁ bp cc ▁launched ▁a 
 ▁1 . 4 ▁p ct ▁to ▁9 . 8 ▁p ct ▁in ▁1986 , ▁bank ▁of 
 ▁planning ▁to ▁shareholders , ▁senior ▁dealer ▁at ▁a ▁cost ▁of ▁about ▁21 ▁m ln ▁ dl rs . 
 ▁the ▁government 's ▁sweeping ▁privat isation ▁programme ▁has ▁also 
 ▁included ▁the ▁sale ▁of ▁saint - gob ain ▁& lt ; sg ep . pa >, ▁and ▁c ie ▁fin anc iere 
 ▁de ▁p ari bas ▁& lt ; p ari . pa >. ▁the ▁sale ▁of ▁the ▁two ▁liquor 
 ▁businesses . ▁it ▁denied ▁a ▁50 ▁p ct ▁rise ▁in ▁plant ings ▁prove 
 ▁true ▁in ▁j une , ▁they ▁said . 
 ▁the ▁dollar ▁opened ▁at ▁a ▁season ally ▁adjusted ▁501 , 000 ▁units ▁after ▁a ▁6 . 5 ▁p ct ▁of ▁the ▁total

100%|██████████| 300/300 [00:01<00:00, 184.23it/s]


Generated Sequence from ['Today'] with temperature 1.0:
Today ▁we ▁still ▁a ▁purchase ▁price ▁for ▁soon ▁about ▁1 - 3 / 8 ▁to ▁one ▁inch ▁in ▁ed monton , 
 ▁chief ▁due ▁next ▁week . ▁ 
 ▁g ord ex ▁said ▁the ▁expansion ▁did ▁indeed ▁rise ▁2 . 
 ▁high - capacity ▁fr duction ▁of ▁1 , 350 , 000 
 ▁ dl rs ▁in ▁quarter ▁and ▁3 , 800 , 000 ▁ dl rs , ▁or ▁ 393 ▁m ln ▁ dl rs 
 ▁or ▁24 ▁c ts ▁a ▁share ▁vs ▁7 . 00 ▁m ln 
 ▁tonnes ▁last ▁month . ▁corn ▁year ▁in ▁gasoline ▁market ▁its ▁value - added ▁19 . 00 ▁ dl rs ▁a 
 ▁barrel . 
 ▁" the ▁agreement ▁which ▁is ▁that ▁sterling ▁have ▁is ▁no ▁new 
 ▁commodity ▁exchange ▁ratio ▁to ▁17 . 2 ▁m ln ▁reported ▁in ▁1985 , ▁the ▁u . s . 
 ▁" these ▁are ▁quota ▁items ▁would ▁not ▁given ▁to ▁the 
 ▁proposed ▁settlement ▁of ▁the ▁liquidity ▁at ▁over ▁this ▁year . 
 ▁be cor 's ▁management ▁and ▁an ▁agreement ▁to ▁stock ▁operations ▁of ▁u . k . 
 ▁k oe hler ▁said ▁in ▁a ▁speech ▁for ▁a ▁due ▁that ▁apply ▁yen ▁in ▁1985 ▁over ▁the 
 ▁meeting ▁on ▁j uly ▁1 , ▁aimin

100%|██████████| 300/300 [00:01<00:00, 173.94it/s]

Generated Sequence from ['Today'] with temperature 1.3:
Today ▁farmers ▁have ▁come ▁meetings ▁with ▁a ▁dollar ▁gr 5 ▁point 
 ▁fell ▁6 . 3 ▁p ct ▁to ▁g np ▁in ▁may , ▁j uly ▁16 ▁and ▁s bc ▁r ▁corp ▁& lt ; t ae >, 
 ▁an ▁economist ▁at ▁a ▁lead ▁conference ▁following ▁the ▁financial ▁markets ▁doing 
 ▁industry ▁analysts ▁paid ▁and ▁be cor able ▁in ▁c etus , ▁taking ▁suit ▁for 
 ▁un ▁840 . 2 ▁m ln ▁vs ▁720 ▁m ln 
 ▁avg ▁sh rs ▁11 . 2 ▁m ln ▁vs ▁11 . 4 ▁m ln 
 ▁extraordinary ▁items ▁debit ▁1 . 1 ▁m ln ▁ dl rs . 
 ▁con ▁give ▁allied - based ▁l fg on - based ▁amb ros iano ▁said ▁that ▁t ok yo ▁has ▁unveiled ▁its ▁fiscal ▁stimulus ▁subsidies ▁of ▁1986 ▁billion ▁yen ▁in ▁the 
 ▁last ▁quarter . 
 ▁this ▁week , ▁including ▁the ▁lou vre ▁meeting ▁advanced ▁in ▁port 
 ▁interest ▁to ▁75 ▁c ts ▁per ▁share ▁for ▁the ▁week ▁ended ▁march ▁28 
 ▁late ▁1989 ▁and ▁maximize ▁a ▁higher ▁margin . ▁st ev ens ▁and ▁technology ▁until 
 ▁investigations ▁are ▁seeking , ▁of ▁up ▁to ▁30 ▁p ct , ▁the ▁ dl rs ▁per ▁t 

### Subword Tokenisation, Dropout = 0.2

In [14]:
# Initialise model
rnn = RNNLanguageModel(
    vocab_size=len(subword_tokenisation_train_vocab),
    embed_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    dropout=0.2,
    pad_idx=subword_tokenisation_train_vocab["<pad>"]
).to(device)

rnn_subword_tokenised_dropout_02_filename = "model_weights_rnn_word_tokens_False_dropout_0.2_20250906-085004"

rnn_subword_tokenised_dropout_02 = load_model(model_initialised=rnn, filename=rnn_subword_tokenised_dropout_02_filename, device=device)

In [15]:
# Cross Entropy Loss
rnn_subword_tokenised_dropout_02_cross_entropy_loss = evaluate_model(
    model=rnn_subword_tokenised_dropout_02,
    converted_tokenised_docs=subword_tokenised_numericalised_docs,
    seq_len=SEQUENCE_LENGTH,
    batch_size=BATCH_SIZE,
    criterion=nn.CrossEntropyLoss(ignore_index=subword_tokenisation_train_vocab["<pad>"]),
    device=device,
    vocab_size=len(subword_tokenisation_train_vocab)
)
print(f"Cross Entropy Loss: {rnn_subword_tokenised_dropout_02_cross_entropy_loss}")

# Perplexity
rnn_subword_tokenised_dropout_02_perplexity = math.exp(rnn_subword_tokenised_dropout_02_cross_entropy_loss)
print(f"Perplexity: {rnn_subword_tokenised_dropout_02_perplexity}")

100%|██████████| 1442/1442 [02:46<00:00,  8.68it/s]


Cross Entropy Loss: 6.820428295001574
Perplexity: 916.3774059070712


In [16]:
# Generate text
starting_sequence = ["Today"]
temperatures = [0.7, 1.0, 1.3]

for temperature in temperatures:
    rnn_subword_tokenised_dropout_02_generated_sequence = generate_text(
        model=rnn_subword_tokenised_dropout_02,
        train_vocab=subword_tokenisation_train_vocab,
        start_seq=starting_sequence,
        temperature=temperature
    )
    print(f"Generated Sequence from {starting_sequence} with temperature {temperature}:")
    print(rnn_subword_tokenised_dropout_02_generated_sequence)
    print()

100%|██████████| 300/300 [00:00<00:00, 328.64it/s]


Generated Sequence from ['Today'] with temperature 0.7:
Today ," ▁he ▁added . 
 ▁" the ▁dollar 's ▁meeting ▁is ▁being ▁placed ▁on ▁the ▁outcome ▁of ▁collective ▁wage ▁agreement . 
 ▁the ▁meeting ▁was ▁attended ▁by ▁farm ▁ministers ▁in ▁the 
 ▁past ▁one ▁days ▁during ▁the ▁year - ago ▁week , ▁traders ▁said . 
 ▁they ▁said ▁the ▁relaxation ▁of ▁controls ▁was ▁now . 
 ▁sharp ▁details ▁were ▁not ▁disclosed . 
 ▁the ▁company , ▁which ▁includes ▁a ▁69 ▁m ln ▁ dl rs ▁for 
 ▁capital ▁spending , ▁less ▁than ▁half ▁of ▁the ▁amount ▁spent ▁70 . 6 ▁p ct ▁from 
 ▁13 . 13 ▁m ln ▁in ▁ap ril , ▁and ▁brought ▁its ▁stake ▁in ▁s ime ▁ dar by , ▁a 
 ▁spokesman ▁for ▁gr aan ▁elevator ▁m ij , ▁the ▁largest ▁employer ▁yesterday . 
 ▁earlier ▁today , ▁the ▁company ▁said ▁the ▁transaction ▁involves ▁the ▁combination ▁of 
 ▁oil ▁and ▁by ▁an ▁existing ▁20 ▁p ct ▁gain ▁in ▁w es ts 
 ▁ uc cel ▁corp ▁as ▁pen ney 's ▁co ▁and 
 ▁& lt ; format ura ▁ ini ez ione ▁po lim eri ▁spa >, ▁gen oa ▁and ▁& lt ; l om > 
 ▁diesel

100%|██████████| 300/300 [00:00<00:00, 375.90it/s]


Generated Sequence from ['Today'] with temperature 1.0:
Today 
 ▁vul ▁prime ▁policy ▁will ▁now ▁be ▁followed ▁by 
 ▁the ▁potential ," ▁said gil lette . 
 ▁shore ▁mentioned ▁half ▁its ▁ch il uba , ▁leader ▁of ▁the ▁z am bian ▁congress ▁in ▁the ▁u . s . 
 ▁carries ▁out ▁a ▁threat ▁to ▁make ▁an ▁effort ▁trade ▁in ▁a ▁near - term ▁strike ▁on 
 ▁t enders , ▁he ▁added ▁in ▁t ok yo ▁were ▁on 
 ▁notice ▁this ▁week ▁because ▁of ▁the ▁j apan ese ▁economy 
 ▁have ▁had ▁been ▁taken ▁higher ▁than ▁the ▁commercial ▁customers . 
 ▁lower ▁taxes ▁would ▁be ▁early ▁by ▁the ▁u . s . 
 ▁currency ▁proposal , ▁which ▁is ▁required ▁to ▁report ▁the ▁ bah rain ▁s mel ter ▁9 . 
 ▁p ea body ▁and ▁co . 
 ▁" there ▁is ▁ongoing , ▁serious ▁thought ▁applied ▁to ▁dome ▁in ▁its ▁retail 
 ▁second ▁offer , ▁thus ▁ass uring ▁said ▁total ▁of ▁118 . 2 ▁m ln ▁s tg 
 ▁the ▁j apan ese ▁early ▁income ▁goods ▁necessary , ▁and ▁that 
 ▁it ▁would ▁be ▁taken ▁with ▁underlying ▁government ▁to ▁keep ▁our ▁trade ▁laws ." 
 ▁" this ▁i

100%|██████████| 300/300 [00:00<00:00, 369.51it/s]

Generated Sequence from ['Today'] with temperature 1.3:
Today 
 ▁also ▁will ▁not ▁do ▁anything ▁for ▁the ▁metal ▁rates . 
 ▁" such ▁a ▁gathering ▁of ▁money ▁and ▁foreign ▁exchange ▁last ▁fixed ▁over ▁& lt ; bank burg 
 ▁pin cus ▁and ▁co ▁& lt ; up cm . o > ▁and ▁a 
 ▁shareholders ▁by ▁the ▁j apan ▁will ▁maintain ▁its ▁prime ▁and 
 ▁both ▁electronic ▁abolish ▁foreign ▁investment ▁in ▁the ▁second ▁six ▁months ▁of ▁1987 . 
 ▁loans ▁to ▁j une ▁17 ▁and ▁112 , 000 ▁b pd , ▁u . s ▁9 ▁and ▁2 . 99 ▁billion ▁ dl rs ▁from ▁86 ▁m ln ▁d lr 
 ▁from ▁one ▁third ▁quarter ▁ended ▁ap ril ▁30 
 ▁shr ▁two . 2 ▁cents ▁with ▁the 
 ▁rice ▁for ▁the ▁energy ▁cocoa ▁in ▁fiscal ▁leading ▁producer 
 ▁are ▁need ▁to ▁put ▁who ▁have ▁some ▁leading ▁industrial ▁house ▁and 
 ▁west ▁ger many 's ▁largely ▁" ▁goal ▁on ▁t uesday 's 
 ▁west ▁g erman ▁monetary ▁policy ▁to ▁aid ▁the ▁dollar - if 
 ▁end ▁and ▁there ▁are ▁are ▁likely ▁at ▁th over ▁banks . 
 ▁" compan ies ▁like ▁dup ont ▁& lt ; dd > 
 ▁ nova ▁said ▁the hor ▁off

## LSTM

### Word Tokenisation, Dropout = 0.0

In [17]:
# Initialise model
lstm = LSTMLanguageModel(
    vocab_size=len(word_tokenisation_train_vocab),
    embed_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    dropout=0.0,
    pad_idx=word_tokenisation_train_vocab["<pad>"]
).to(device)

lstm_word_tokenised_dropout_0_filename = "model_weights_lstm_word_tokens_True_dropout_0.0_20250905-094011"

lstm_word_tokenised_dropout_0 = load_model(model_initialised=lstm, filename=lstm_word_tokenised_dropout_0_filename, device=device)

In [18]:
# Cross Entropy Loss
lstm_word_tokenised_dropout_0_cross_entropy_loss = evaluate_model(
    model=lstm_word_tokenised_dropout_0,
    converted_tokenised_docs=word_tokenised_numericalised_docs,
    seq_len=SEQUENCE_LENGTH,
    batch_size=BATCH_SIZE,
    criterion=nn.CrossEntropyLoss(ignore_index=word_tokenisation_train_vocab["<pad>"]),
    device=device,
    vocab_size=len(word_tokenisation_train_vocab)
)
print(f"Cross Entropy Loss: {lstm_word_tokenised_dropout_0_cross_entropy_loss}")

# Perplexity
lstm_word_tokenised_dropout_0_perplexity = math.exp(lstm_word_tokenised_dropout_0_cross_entropy_loss)
print(f"Perplexity: {lstm_word_tokenised_dropout_0_perplexity}")

100%|██████████| 1073/1073 [03:27<00:00,  5.17it/s]

Cross Entropy Loss: 11.687003160761765
Perplexity: 119014.80317215814


In [19]:
# Generate text
starting_sequence = ["Today"]
temperatures = [0.7, 1.0, 1.3]

for temperature in temperatures:
    lstm_word_tokenised_dropout_0_generated_sequence = generate_text(
        model=lstm_word_tokenised_dropout_0,
        train_vocab=word_tokenisation_train_vocab,
        start_seq=starting_sequence,
        temperature=temperature
    )
    print(f"Generated Sequence from {starting_sequence} with temperature {temperature}:")
    print(lstm_word_tokenised_dropout_0_generated_sequence)
    print()

  4%|▍         | 13/300 [00:00<00:15, 18.65it/s]


Generated Sequence from ['Today'] with temperature 0.7:
Today that the federal reserve will first determine from june , it said .



100%|██████████| 300/300 [00:02<00:00, 148.60it/s]


Generated Sequence from ['Today'] with temperature 1.0:
Today quoted central officials as saying with 60 . 9 mln hectares of wheat were affected in london . yesterday , first four spokesman said . but he added there was a general company , told reuters the company was examining potential candidates but not yet talking with any . he declined to name the companies but said a deal could occur as soon as the second half of 1987 . phelps dodge , which ranks as the largest copper producer in the last day of its latest offer was not immediately . his sold agreed to merge , forming to be named , said . he said the ibc is not related from a . h . robins co said its royal airlines inc , which serves a special position , he terminated of the two other striking locals , representing 600 office and technical workers . production at trail and kimberley has been shut down since the strike started may 9 , this last year , after importing between one oil output in calendar 1986 was more than 6 . 13 cen

100%|██████████| 300/300 [00:01<00:00, 156.17it/s]

Generated Sequence from ['Today'] with temperature 1.3:
Today despite cutbacks in output in the u . k . and japanese soviet ships have also been lost , with heavy problems and was in the best long-term interests of its 38 mln dlr per share merger ' company common shares with stock at the 1 . 45 dlrs and 3 . 51 dlrs per share on at the end of dec 17 , 1986 , and profits of its product , which was halted on october this year . but for the start of the bundesbank is not firming . . . and broad moves which have been re-registered to canadian farm money supply for dome and its 18 and payment of u . s . international manufacturing industries to continue their purchases after improving on the southern eastern of march . but most dealers said the position of the deficit reduction goal were around anything but you can done in west investment fund ( ciif ) and 1 . 25 mln a year loss were higher than after taxes . as banks are expected to be resumed at least first loss carryforward . china canada

### Word Tokenisation, Dropout = 0.2

In [20]:
# Initialise model
lstm = LSTMLanguageModel(
    vocab_size=len(word_tokenisation_train_vocab),
    embed_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    dropout=0.2,
    pad_idx=word_tokenisation_train_vocab["<pad>"]
).to(device)

lstm_word_tokenised_dropout_02_filename = "model_weights_lstm_word_tokens_True_dropout_0.2_20250906-071938"

lstm_word_tokenised_dropout_02 = load_model(model_initialised=lstm, filename=lstm_word_tokenised_dropout_02_filename, device=device)

In [21]:
# Cross Entropy Loss
lstm_word_tokenised_dropout_02_cross_entropy_loss = evaluate_model(
    model=lstm_word_tokenised_dropout_02,
    converted_tokenised_docs=word_tokenised_numericalised_docs,
    seq_len=SEQUENCE_LENGTH,
    batch_size=BATCH_SIZE,
    criterion=nn.CrossEntropyLoss(ignore_index=word_tokenisation_train_vocab["<pad>"]),
    device=device,
    vocab_size=len(word_tokenisation_train_vocab)
)
print(f"Cross Entropy Loss: {lstm_word_tokenised_dropout_02_cross_entropy_loss}")

# Perplexity
lstm_word_tokenised_dropout_02_perplexity = math.exp(lstm_word_tokenised_dropout_02_cross_entropy_loss)
print(f"Perplexity: {lstm_word_tokenised_dropout_02_perplexity}")

100%|██████████| 1073/1073 [02:22<00:00,  7.52it/s]

Cross Entropy Loss: 9.47497796895675
Perplexity: 13029.586940404579


In [22]:
# Generate text
starting_sequence = ["Today"]
temperatures = [0.7, 1.0, 1.3]

for temperature in temperatures:
    lstm_word_tokenised_dropout_02_generated_sequence = generate_text(
        model=lstm_word_tokenised_dropout_02,
        train_vocab=word_tokenisation_train_vocab,
        start_seq=starting_sequence,
        temperature=temperature
    )
    print(f"Generated Sequence from {starting_sequence} with temperature {temperature}:")
    print(lstm_word_tokenised_dropout_02_generated_sequence)
    print()

100%|██████████| 300/300 [00:01<00:00, 207.82it/s]


Generated Sequence from ['Today'] with temperature 0.7:
Today was not in the best interests of the united states and japan to stabilise exchange rates . in the debtor countries , the organisation ' s budget deficit and the further build-up of an already-substantial net external asset position could well lead to upward pressure on the yen . it said that earlier tightening growth was high by the fall in the industrial nations . in the past six months ago other than the most unchanged seen the dollar fall had been temporarily is recognized after the administration ( stoltenberg ) said . naturally , we welcomed the defeat but there is a chance the u . s . congress has been widely interpreted as a result , he said . i think the stock market ' s biggest sales was now to reduce tariffs on japanese electronic imports and a slow turnaround in the second half of 1986 . atlas said negotiations were continuing on the acquisition by bond of the company ' s existing bank loans and their restructurin

100%|██████████| 300/300 [00:00<00:00, 354.13it/s]


Generated Sequence from ['Today'] with temperature 1.0:
Today at 17 . 47 dlrs per barrel in april from a high almost 14 pct decline to the dollar but it is quite too early to venture out of a small position of the key run , but others may have to influence the industry of the west german economy , the organisation for economic cooperation and development ( oecd ) said here . the economic situation has deteriorated in recent months , and very recession , said he expects the dollar to hold steady against the mark and swiss franc but to weaken further against the yen , possibly by prime minister yasuhiro lampe . 00 interest rate differentials half may be 8 . 5 pct in 1986 . inflation depends on its 1986 crop for profits ended april 8 , 1987 , it was warm with some rainfall , which provided the beet growth , government officials said . theys said the plant would cost an estimated 270 mln canadian francs . the group had a 10 . 5 pct stake under coffee monday , by three of a series of proble

100%|██████████| 300/300 [00:00<00:00, 391.24it/s]

Generated Sequence from ['Today'] with temperature 1.3:
Today for a gradually which would watch the matter as long as the uncertainty will accept the contract pact . according to a copy of perelman ' s letter released by gillette , but that the chances of are seen as a very important planning policy . it said the authorities is prepared to make another large non-copper acquisition on the way . after the trade figures were available on buffer stock repurchase agreements , traders said . the dollar opened at similar levels at pct against the dollar dropped last week . he said sterling bank had not risen some cuts . poehl said west german beer and small borrowings by the end of the week , just up around 3-7/8 pct . saudi activity was also talking with the new exchange as opec repurchase agreements it helped stimulate its economic wheat on oils and fats , parliamentarians said . a broad cross-section of members of parliament voiced their opposition to the tax during a debate in south afric

### Subword Tokenisation, Dropout = 0.0

In [23]:
# Initialise model
lstm = LSTMLanguageModel(
    vocab_size=len(subword_tokenisation_train_vocab),
    embed_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    dropout=0.0,
    pad_idx=subword_tokenisation_train_vocab["<pad>"]
).to(device)

lstm_subword_tokenised_dropout_0_filename = "model_weights_lstm_word_tokens_False_dropout_0.0_20250906-081655"

lstm_subword_tokenised_dropout_0 = load_model(model_initialised=lstm, filename=lstm_subword_tokenised_dropout_0_filename, device=device)

In [24]:
# Cross Entropy Loss
lstm_subword_tokenised_dropout_0_cross_entropy_loss = evaluate_model(
    model=lstm_subword_tokenised_dropout_0,
    converted_tokenised_docs=subword_tokenised_numericalised_docs,
    seq_len=SEQUENCE_LENGTH,
    batch_size=BATCH_SIZE,
    criterion=nn.CrossEntropyLoss(ignore_index=subword_tokenisation_train_vocab["<pad>"]),
    device=device,
    vocab_size=len(subword_tokenisation_train_vocab)
)
print(f"Cross Entropy Loss: {lstm_subword_tokenised_dropout_0_cross_entropy_loss}")

# Perplexity
lstm_subword_tokenised_dropout_0_perplexity = math.exp(lstm_subword_tokenised_dropout_0_cross_entropy_loss)
print(f"Perplexity: {lstm_subword_tokenised_dropout_0_perplexity}")

100%|██████████| 1442/1442 [01:43<00:00, 13.88it/s]

Cross Entropy Loss: 10.090113407943848
Perplexity: 24103.525809895324


In [25]:
# Generate text
starting_sequence = ["Today"]
temperatures = [0.7, 1.0, 1.3]

for temperature in temperatures:
    lstm_subword_tokenised_dropout_0_generated_sequence = generate_text(
        model=lstm_subword_tokenised_dropout_0,
        train_vocab=subword_tokenisation_train_vocab,
        start_seq=starting_sequence,
        temperature=temperature
    )
    print(f"Generated Sequence from {starting_sequence} with temperature {temperature}:")
    print(lstm_subword_tokenised_dropout_0_generated_sequence)
    print()

100%|██████████| 300/300 [00:01<00:00, 222.37it/s]


Generated Sequence from ['Today'] with temperature 0.7:
Today ▁and ▁ meyer ▁detective 
 ▁agency ▁inc ▁of ▁national ▁park , ▁n . j . 
 ▁the ▁company ▁said ▁it ▁is ▁evaluating ▁the ▁proposals ▁from ▁p aul 
 ▁bil ze rian ▁and ▁from ▁a ▁third ▁party ▁which ▁is ▁active ▁in ▁the 
 ▁leve raged ▁bu yout ▁field ▁but ▁which ▁pay ▁n ' ▁ pak ▁declined ▁to 
 ▁identify . 
 ▁it ▁said ▁the ▁bil ze rian ▁proposal ▁calls ▁for ▁distribution ▁of 
 ▁stock holders ▁as ▁a ▁result ▁of ▁its ▁office ▁credit ▁for ▁a ▁total ▁of ▁50 . 9 
 ▁m ln ▁ dl rs ▁and ▁net ▁yield ▁2 . 99 ▁p ct . ▁if ▁b razil ▁does ▁not ▁resume 
 ▁payments , ▁1987 ▁net ▁would ▁be ▁cut ▁by ▁72 ▁m ln ▁ dl rs , ▁would 
 ▁be ▁full ▁impact ▁of ▁the ▁consumer ▁exploration 
 ▁over ▁the ▁first ▁10 ▁years ▁are ▁holding . ▁these ▁markets ▁are 
 ▁underway ▁to ▁give ▁a ▁forecast ▁for ▁the ▁six ▁months . 
 ▁c ondon ▁attributed ▁the ▁fed ▁would ▁not ▁cause ▁any ▁action 
 ▁within ▁its ▁own ▁tender ▁lead ▁for ▁the ▁u . s . ▁and ▁ eur ope ▁seek 
 ▁removal ▁of

100%|██████████| 300/300 [00:01<00:00, 293.76it/s]


Generated Sequence from ['Today'] with temperature 1.0:
Today ▁and ▁pipe 
 ▁32 ▁times ▁ han over , ▁shopping ▁re alty ▁& lt ; uk . as >, 
 ▁said ▁au str alia 's ▁two ▁largest ▁wheat 
 ▁( excluding ▁temples ) ▁will ▁jointly ▁as ▁a ▁long - time ▁span , ▁bankers 
 ▁said . 
 ▁" we 're ▁a ▁net ▁an ▁improvement ▁of ▁six ▁people ," ▁he ▁told 
 ▁re uters . 
 ▁" a ▁heavy ▁heart ," ▁meaning ▁that ▁he ▁would ▁take ▁it ?" ▁he ▁added . 
 ▁the ▁o ecd ▁said ▁that ▁the ▁dividend ▁was ▁the ▁third ▁consecutive 
 ▁days ▁of ▁a ▁continuing ▁attempt ▁on ▁producing ▁trade ▁im bal ances , 
 ▁cl ark ▁told ▁a ▁meeting ▁of ▁the ▁minerals ▁research ▁organisation 
 ▁in ▁m ilton ▁ke ynes ▁he ▁fore saw ▁a ▁250 , 000 ▁ounce ▁increase ▁in ▁may 
 ▁s audi ▁ ara bia 's ▁government ▁ahead ▁of 
 ▁schedule . 
 ▁it ▁added ▁the ▁market ▁share ▁was ▁acquired ▁2 . 5 ▁m ln ▁shares ▁of ▁v ira tek 
 ▁common . ▁to ▁date , ▁i cn ▁said ▁it ▁bought ▁a ▁total ▁of ▁about ▁1 . 2 ▁m ln 
 ▁shares ▁of ▁v ira tek , ▁including ▁the ▁shares ▁p

100%|██████████| 300/300 [00:00<00:00, 354.89it/s]

Generated Sequence from ['Today'] with temperature 1.3:
Today ▁fits ▁to ▁prevent ▁any ▁outbreaks ▁of ▁ ri oting 
 ▁ ku wa iti ▁d ▁co ▁l td ▁of ▁which ▁t nt 100 ▁will ▁be 
 ▁upgraded ▁to ▁around ▁one ▁m ln ▁tonnes ▁a ▁year 's ▁45 , ▁government ▁spokesman ▁fried helm 
 ▁l td ▁( 5 ) ▁in ▁new ▁y ork ▁and ▁142 . 50 ▁to ▁a ▁prime ▁rate ▁to ▁17 ▁p ct . 
 ▁" this ▁per ▁" by ▁bri tain ▁has ▁threatened ▁to ▁damage ▁from ▁i raq , ▁the ▁u . s . 
 ▁were ▁to ▁58 . 0 ▁billion ▁ dl rs ▁and ▁imports ▁rose ▁only ▁more ▁than ▁two ▁m ln 
 ▁ dl rs ▁from ▁the ▁fed ▁averaged ▁20 ▁ dl rs ▁a ▁week 
 ▁of ▁dup ont 's ▁con oco ▁buy ▁shares ▁he ▁sold ▁chief ▁ro bert ▁de ▁la ▁gar za ▁has 
 ▁said ▁the ▁committee ▁working ▁council ▁documents ▁concerned 
 ▁the ▁nation 's ▁decision ▁to ▁continued ▁higher ▁interest ▁rates 
 ▁more ▁than ▁capital izing ▁prices ▁to ▁previous ▁short - term 
 ▁deposits ▁and ▁30 ▁p ct ▁on ▁ap ril ▁17 . ▁" the ▁world 's ▁largest ▁debtor ▁corporation , 
 ▁has ▁been ▁making ▁of ▁over ▁three ▁p c

### Subword Tokenisation, Dropout = 0.2

In [26]:
# Initialise model
lstm = LSTMLanguageModel(
    vocab_size=len(subword_tokenisation_train_vocab),
    embed_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    dropout=0.2,
    pad_idx=subword_tokenisation_train_vocab["<pad>"]
).to(device)

lstm_subword_tokenised_dropout_02_filename = "model_weights_lstm_word_tokens_False_dropout_0.2_20250906-092405"

lstm_subword_tokenised_dropout_02 = load_model(model_initialised=lstm, filename=lstm_subword_tokenised_dropout_02_filename, device=device)

In [27]:
# Cross Entropy Loss
lstm_subword_tokenised_dropout_02_cross_entropy_loss = evaluate_model(
    model=lstm_subword_tokenised_dropout_02,
    converted_tokenised_docs=subword_tokenised_numericalised_docs,
    seq_len=SEQUENCE_LENGTH,
    batch_size=BATCH_SIZE,
    criterion=nn.CrossEntropyLoss(ignore_index=subword_tokenisation_train_vocab["<pad>"]),
    device=device,
    vocab_size=len(subword_tokenisation_train_vocab)
)
print(f"Cross Entropy Loss: {lstm_subword_tokenised_dropout_02_cross_entropy_loss}")

# Perplexity
lstm_subword_tokenised_dropout_02_perplexity = math.exp(lstm_subword_tokenised_dropout_02_cross_entropy_loss)
print(f"Perplexity: {lstm_subword_tokenised_dropout_02_perplexity}")

100%|██████████| 1442/1442 [01:45<00:00, 13.69it/s]


Cross Entropy Loss: 7.988271788982016
Perplexity: 2946.200900362328


In [28]:
# Generate text
starting_sequence = ["Today"]
temperatures = [0.7, 1.0, 1.3]

for temperature in temperatures:
    lstm_subword_tokenised_dropout_02_generated_sequence = generate_text(
        model=lstm_subword_tokenised_dropout_02,
        train_vocab=subword_tokenisation_train_vocab,
        start_seq=starting_sequence,
        temperature=temperature
    )
    print(f"Generated Sequence from {starting_sequence} with temperature {temperature}:")
    print(lstm_subword_tokenised_dropout_02_generated_sequence)
    print()

100%|██████████| 300/300 [00:01<00:00, 231.91it/s]


Generated Sequence from ['Today'] with temperature 0.7:
Today ," ▁said ▁a ▁wall ▁street ▁arbit rag eur . 
 ▁but ▁he ▁said ▁the ▁bank ▁regards ▁the ▁over draft 
 ▁reference ▁rate , ▁based ▁on ▁short - term ▁rate ▁trends , ▁as ▁its ▁key 
 ▁prime ▁lending ▁rate ▁to ▁corporate ▁customers . ▁the ▁loan ▁reference 
 ▁rate ▁is ▁based ▁on ▁longer ▁term ▁trends . 
 ▁the ▁bank ▁is ▁the ▁latest ▁to ▁cut ▁prime ▁rates ▁in ▁the ▁next ▁few ▁weeks . 
 ▁given ▁an ▁average ▁yield ▁of ▁1 . 09 ▁billion ▁ dl rs ▁of ▁j uly ▁accounting ▁to 
 ▁three ▁m ln ▁ dl rs ▁to ▁226 . 4 ▁m ln ▁ dl rs , ▁compared ▁with ▁169 . 2 ▁m ln ▁ dl rs 
 ▁in ▁year ▁to ▁de cember ▁31 , ▁1986 . 
 ▁shr ▁18 . 9 ▁m ln ▁vs ▁5 . 7 ▁m ln 
 ▁note ▁- ▁full ▁name ▁is ▁data ▁services , ▁inc ▁said ▁it ▁will 
 ▁offer ▁5 . 0 ▁m ln ▁ dl rs ▁cash ▁of ▁2 . 2 ▁m ln ▁common ▁shares ▁of ▁stock ▁for ▁each ▁ uc cel 
 ▁wholly ▁based ▁on ▁50 ▁p ct ▁of ▁the ▁equity ▁in ▁j une , ▁it ▁added . 
 ▁the ▁application ▁of ▁the ▁additional ▁48 ▁p ct ▁since ▁the 
 ▁t

100%|██████████| 300/300 [00:00<00:00, 353.21it/s]


Generated Sequence from ['Today'] with temperature 1.0:
Today >. 
 ▁j apan 's ▁letter ▁follows ▁a ▁12 ▁p ct ▁increase ▁in ▁s ept ember ▁single - family ▁unit ▁starts ▁a ▁1 . 5 ▁p ct ▁to ▁1 . 73 
 ▁m ln ▁s tg , ▁it ▁is ▁also ▁most ▁for ▁good ▁recovery , ▁c omin co ▁said ▁in 
 ▁view ▁that ▁levels ▁should ▁be ▁at ▁least ▁behind ▁this ▁time . 
 ▁the ▁officials ▁said ▁ger many ▁had ▁practically ▁no ▁growth ▁in ▁the ▁long ▁term 
 ▁and ▁bond ▁prices ▁was ▁not ▁immediately ▁clear ▁the ▁company 
 ▁will ▁be ▁able ▁to ▁offer ▁specifics ▁from ▁the ▁sale ▁of ▁a ▁controlling ▁of 
 ▁its ▁reliance ▁standard ▁life 
 ▁insurance ▁co . ▁a ▁d alian ▁subsidiary , ▁r - held ▁dat eline ron 
 ▁equipment ▁to ▁carry lift ▁as ▁s ears ▁ roe buck ▁and ▁co 
 ▁& lt ; s > ▁converted ▁subsidiary ▁senior ▁management ▁of 
 ▁international ▁data ▁corp ▁said ▁it ▁has ▁agreed ▁to 
 ▁combine ▁its ▁cocoa ▁processing ▁businesses ▁with ▁those ▁of ▁s . ▁ . c . ▁terms , 
 ▁effective ▁yesterday , ▁within ▁400 ▁miles ▁east ▁of ▁v an

100%|██████████| 300/300 [00:00<00:00, 372.94it/s]

Generated Sequence from ['Today'] with temperature 1.3:
Today 's ▁further ▁cut ▁since ▁they ▁should ▁earn ▁more ▁cash 
 ▁or ▁before ▁in ▁fiscal ▁production ▁came . 
 ▁speaking ▁at ▁a ▁forum ▁for ▁indust ria lists ▁of ▁major ▁central ▁banks ▁meeting 
 ▁today 's ▁federal ▁reserve . 
 ▁the ▁news ▁eroded ▁the ▁most ▁immediate ▁to ▁over ▁the ▁r eval uation 
 ▁general ▁meeting , ▁the ▁meat ▁investment ▁firm ▁said . 
 ▁sci - med ▁said ▁it ▁continues ▁to ▁be ▁identified ▁affected ▁by ▁ ron ald 
 ▁per elman , ▁offered ▁20 ▁ dl rs ▁lower ▁than ▁an ▁average ▁yield ▁1 . 04 ▁ dl rs ▁in ▁liqu idation 
 ▁value ▁of ▁a ▁stock ▁in ▁each ▁share ▁of ▁arrays ▁bought ▁or ▁less ▁than 
 ▁2 . 4 ▁in ▁response ▁to ▁its ▁oil ▁market ." 
 ▁ donald ▁trump , ▁rose ▁to ▁1 . 0 ▁m ln ▁ dl rs ▁reflecting ▁proposed ▁to 
 ▁hol stein / 1 , 000 ▁when ▁h ilton ▁petroleum ▁spending ▁mill ▁workers 
 ▁at ▁one ▁of ▁its ▁products , ▁at las ▁group , ▁prices ▁collapsed ▁by ▁the ▁end 
 ▁of ▁an ▁oil ▁industry . ▁however , ▁government